In [28]:
#Dont change anything here
import numpy as np
from keras.callbacks import ModelCheckpoint
import keras
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import Adam
import matplotlib.pyplot as plt


class GeneralModel(object):
    
    def __init__(self):
        pass
    
    def set_parameters(self, batch_size = 500, epoch = 6):
        self.batch_size = batch_size
        self.epoch = epoch
        
    def set_training_data (self, X_train,x_height, x_width,channel, Y_train, num_class = 10):# give data in numpy and y in onehot 
        self.X_train = X_train
        self.Y_train = Y_train
        self.num_class = num_class
        self.x_height = x_height
        self.x_width = x_width
        self.channel = channel
        
    def set_test_data (self, X_test, Y_test):
        self.Y_test = Y_test
        self.X_test = X_test
        
    def get_standard_model(self, numb_1st_layer_kernam = 96, number_other_lyer_kernal = 96):
        model = Sequential()
        # model.add(Conv2D(10, (11,11), strides=(4,4), activation='relu', padding='valid', input_shape=(img_height, img_width, channel,)))
        # for original Alexnet
        model.add(Conv2D(numb_1st_layer_kernam, (11,11), strides=(4,4), padding='same', input_shape=(self.x_height, self.x_width, self.channel,)))
        convout1 = Activation('relu')
        model.add(convout1)
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
        # Local Response normalization for Original Alexnet
        model.add(BatchNormalization())

        #model.add(Conv2D(2, (5,5), activation='relu', padding='same'))
        model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
        # Local Response normalization for Original Alexnet
        model.add(BatchNormalization())

        #model.add(Conv2D(3, (3,3), activation='relu', padding='same'))
        #model.add(Conv2D(3, (3,3), activation='relu', padding='same'))
        model.add(Conv2D(3, (3,3), activation='relu', padding='same'))
        #model.add(MaxPooling2D(pool_size=(3, 3), strides=(2,2)))
        # Local Response normalization for Original Alexnet
        model.add(BatchNormalization())

        model.add(Flatten())
        model.add(Dense(100, activation='relu'))
        model.add(Dropout(0.2))
        #model.add(Dense(4096, activation='relu'))
        #model.add(Dropout(0.5))
        model.add(Dense(self.num_class, activation='softmax'))
        model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

        
        return model
    
    def set_chekpointer(self, checkpointer):
        self.checkpointer = checkpointer
        
    def model_fit(self, model):
        self.model = model
        self.history = self.model.fit(self.X_train, self.Y_train,
                              batch_size = self.batch_size,
                              epochs=self.epoch,
                              callbacks=self.checkpointer,
                              verbose=1,
                              validation_data = (self.X_test, self.Y_test))

        return self.model
    
    def plot_accuracy(self, directory):
        plt.plot(self.history.history['acc'])
        plt.plot(self.history.history['val_acc'])
        plt.title('model accuracy')
        plt.ylabel('accuracy')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.savefig(directory)
        
    def plot_loss(self, directory):
        plt.plot(self.history.history['loss'])
        plt.plot(self.history.history['val_loss'])
        plt.title('model loss')
        plt.ylabel('loss')
        plt.xlabel('epoch')
        plt.legend(['train', 'test'], loc='upper left')
        plt.savefig(directory)
        
    def model_evaluate(self):
        self.score = self.model.evaluate(self.X_test, self.Y_test, verbose=0)
        return self.score
    
    

In [31]:
#Generate a model instance 
import os
subtask2_path = 'Subtask2/'

if not os.path.exists(subtask2_path):
    os.makedirs(subtask2_path)
    
    
mymodel = GeneralModel()
#set parameters 
mymodel.set_parameters(batch_size =300,epoch = 5)
num_classes  = 10
#give training and test data 

X = np.load('X.npy')
Y = np.load('Y.npy')

(x_train, y_train) = (X[0:15000], Y[0:15000])
(x_test, y_test) = (X[15000:], Y[15000:])
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]
print(img_height, img_width, channel)
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

#set_training_data (self, X_train,x_height, x_width,channel, Y_train, num_class = 10)
mymodel.set_training_data(x_train,img_height,img_width,channel,y_train,num_classes)
mymodel.set_test_data (x_test, y_test)



best_weight_path = subtask2_path+"best_weights.hdf5"

early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss',
                              min_delta=0,
                              patience=2,
                              verbose=0, mode='auto')
    
checkpointer = ModelCheckpoint(filepath="best_weights.hdf5",
                               monitor='val_acc',
                               verbose=1,
                               save_best_only= True)
mymodel.set_chekpointer([checkpointer,early_stopping])

model = mymodel.get_standard_model()

model = mymodel.model_fit(model)
#'vizualize(self,layer_num, num_kernel,gray=False, gaus_filter=True,sigma = 1,shapeofgrid=(16,8) ):

import DataViz as dv
keranl_vix =  dv.DataVizulaize(model)
#for layer in model.layers:
#   weights = layer.get_weights() # list of numpy arrays
vizul = keranl_vix.vizualize(0,96, shapeofgrid = (8,16),gray=True)




227 227 3
(15000, 10)
Train on 15000 samples, validate on 7424 samples
Epoch 1/5
  300/15000 [..............................] - ETA: 17:05 - loss: 3.2064 - acc: 0.0800

KeyboardInterrupt: 